In [ ]:
!pip3 install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
DATAPATH = "/content/drive/MyDrive/CS779/ILDC_single/ILDC_single/ILDC_single.csv/ILDC_single.csv"
data = pd.read_csv(DATAPATH)
data.head()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, LongformerModel
from transformers import LongformerTokenizerFast

checkpoint = "allenai/longformer-base-4096"
model = LongformerModel.from_pretrained(checkpoint)
tokenizer = LongformerTokenizerFast.from_pretrained(checkpoint)

tokens = tokenizer(data['text'][0], truncation = True, return_tensors = 'pt')['input_ids']

print(tokens)
print(tokens.shape)

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[  0, 274,   4,  ...,   4,  20,   2]])
torch.Size([1, 4096])


In [ ]:
print(tokenizer.model_max_length)
print(tokenizer.all_special_tokens)
print(tokenizer.num_special_tokens_to_add())

4096
['<s>', '</s>', '<unk>', '<pad>', '<mask>']
2


In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
sentences = sent_tokenize(data['text'][0])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
print(sentences[:10])
print(len(sentences))
max_tokens_in_sentence = max([len(tokenizer.tokenize(sentence)) for sentence in sentences])
print(max_tokens_in_sentence)

[' F. NARIMAN, J.', 'Leave granted.', 'In 2008, the Punjab State Water Supply Sewerage Board, Bhatinda issued numberice inviting tender for extension and augmentation of water supply, sewerage scheme, pumping station and sewerage treatment plant for various towns mentioned therein on a turnkey basis.', 'On 25.9.2008, the appellant companypany, which is Signature Not Verified involved in civil electrical works in India, was awarded the said Digitally signed by NIDHI AHUJA Date 2019.03.11 173359 IST Reason tender after having been found to be the best suited for the task.', 'On 16.1.2009, a formal companytract was entered into between the appellant and respondent No.', '2.', 'It may be mentioned that the numberice inviting tender formed part and parcel of the formal agreement.', 'Contained in the numberice inviting tender is a detailed arbitration clause.', 'In this matter, we are companycerned with clause 25 viii  which is set out as follows- viii.', 'It shall be an essential term of th

In [ ]:
# initialize
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(sentences) - 1:
      chunks.append(chunk.strip()) # save the chunk
    
  else: 
    chunks.append(chunk.strip()) # save the chunk
    
    # reset 
    length = 0 
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

2

In [ ]:
print([len(tokenizer.tokenize(c)) for c in chunks])
print([len(tokenizer(c).input_ids) for c in chunks])

[4087, 1738]
[4089, 1740]


In [ ]:
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

In [ ]:
for input in inputs:
  output = model.generate(**input)
  print(tokenizer.decode(*output, skip_special_tokens=True))

TypeError: ignored

In [ ]:


# default is local attention everywhere
# the forward method will automatically set global attention on question tokens
attention_mask = encoding["attention_mask"]

start_scores, end_scores = model(input_ids, attention_mask=attention_mask)
all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0].tolist())

answer_tokens = all_tokens[torch.argmax(start_scores):torch.argmax(end_scores)+1]
answer = tokenizer.decode(tokenizer.convert_tokens_to_ids(answer_tokens))

# output => democratized NLP


TypeError: ignored